# Métodos de suavizado de series de tiempo
## Promedio móvil
El promedio móvil se construye sustituyendo cada valor de una serie por la media obtenida con esa observación, y valores anteriores y posteriores

![Promedio móvil](promov.png)

## Promedio móvil ponderado
Para mostrar el uso de éste método, se utilizará la primera parte del ejemplo anterior de la venta de gasolina. El método consiste en asignar un factor de ponderación distinto para cada dato. Generalmente, a la observación o dato más reciente a partir del que se quiere hacer el pronóstico, se le asigna el mayor peso, y este peso disminuye en los valores de datos más antiguos.

![Promedio móvil ponderado](promovpon.png)

## Suavizamiento exponencial
El suavizamiento exponencial emplea un promedio ponderado de la serie de tiempo pasada como pronóstico; es un caso especial del método de promedios móviles ponderados en el cual sólo se selecciona un peso o factor de ponderación: el de la observación más reciente. En la práctica comenzamos haciendo que F1, el primer valor de la serie de valores uniformados, sea igual a $Y_1$.

$F_{t+1}=\alpha Y_{t}(1-\alpha)F_1$

![Suavizado](suavizamiento.png)

In [ ]:
# Importar los modulos necesarios
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
# Cargar los datos
url='https://raw.githubusercontent.com/JASDataCTG/Diplomado-ML/main/Modulo%209/Datasets/pasajeros.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
df.info()

In [ ]:
df['mes'] = pd.to_datetime(df['mes'])
df.dtypes

In [ ]:
# Visualización
plt.figure(figsize=(15,8))
plt.plot(df['mes'],df['pasajeros'],'go:',markerfacecolor='w')

plt.legend(['Datos reales'])
plt.show()

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y validación
date = '2016-12-01'
df_train = df[df['mes'] <= datetime.datetime.strptime(date,'%Y-%m-%d')]
df_test = df[df['mes'] > datetime.datetime.strptime(date,'%Y-%m-%d')]

In [ ]:
df_train.tail()

In [ ]:
df_test.head()

In [ ]:
# visualzation
plt.figure(figsize=(15,8))
plt.plot(df_train['mes'],df_train['pasajeros'],'go:',markerfacecolor='w')
plt.plot(df_test['mes'],df_test['pasajeros'],'bo:',markerfacecolor='w')

plt.legend(['Datos de entrenamiento','Datos de validación'])
plt.show()

# Pronósticos utilizando métodos de suavizado
## Naive forecasting
$Y_{t+1} = Y_t$

In [ ]:
df_train['valor_ajustado'] = df_train['pasajeros'].shift()

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
df_test['prediccion'] = 67983
df_test.head()

In [ ]:
# Calcular el margen de error
error = df_train['pasajeros'] - df_train['valor_ajustado']
me = 1.96*error.std()

In [ ]:
# Visualización del modelo
plt.figure(figsize=(15,8))
plt.plot(df_train['mes'],df_train['pasajeros'],'go:',markerfacecolor='w')
# Valores ajustados
plt.plot(df_train['mes'],df_train['valor_ajustado'],'s:',color='orange',markerfacecolor='w')


plt.plot(df_test['mes'],df_test['pasajeros'],'bo:',markerfacecolor='w')
# Predicción
plt.plot(df_test['mes'],df_test['prediccion'],'s:',color='red',markerfacecolor='w')

# Mostrar intervalo de confianza
plt.fill_between(df_test['mes'],(df_test['prediccion']+me),(df_test['prediccion']-me),alpha=0.1)
plt.legend(['Datos de entrenamiento','Datos ajustados','Datos de validación','Predicción','Intervalo de confianza'])

# Escalar ejes
start_date = datetime.datetime.strptime("2015-01-01","%Y-%m-%d")
end_date = datetime.datetime.strptime("2018-01-01","%Y-%m-%d")
plt.axis(xmin=start_date,
         xmax=end_date)
plt.show()


## Media móvil (SMA) de 3 periodos

In [ ]:
df_train_sma = df_train[['mes','pasajeros']]
df_test_sma = df_test[['mes','pasajeros']]

In [ ]:
df_train_sma.head()

In [ ]:
df_train_sma['ajust_3'] = df_train_sma.rolling(window=3).mean() # mean, median, std, var

In [ ]:
df_train_sma.tail()

In [ ]:
df_test_sma['prediccion'] = 68037.666667

In [ ]:
# Calcular el margen de error
error_3 = df_train_sma['pasajeros'] - df_train_sma['ajust_3']
me_3 = 1.96*error_3.std()
# Visalización
plt.figure(figsize=(15,8))
plt.plot(df_train_sma['mes'],df_train_sma['pasajeros'],'bo:',markerfacecolor='w')
# Datos de entrenamiento
plt.plot(df_train_sma['mes'],df_train_sma['ajust_3'],'s:',color='orange',markerfacecolor='w')


plt.plot(df_test_sma['mes'],df_test_sma['pasajeros'],'go:',markerfacecolor='w')
# Predicciones
plt.plot(df_test_sma['mes'],df_test_sma['prediccion'],'s:',color='red',markerfacecolor='w')

# Intervalo de confianza
plt.fill_between(df_test_sma['mes'],(df_test_sma['prediccion']+me_3),(df_test_sma['prediccion']-me_3),alpha=0.1)
plt.legend(['Datos de entrenamiento','Datos ajustados','Datos de validación','Predicción','Intervalo de confianza'])

# Escalado
# start_date = datetime.datetime.strptime("2015-01-01","%Y-%m-%d")
# end_date = datetime.datetime.strptime("2018-01-01","%Y-%m-%d")
# plt.axis(xmin=start_date,
#          xmax=end_date)
plt.show()

## Media móvil $q$ periodos
El paramétro q se evidencia en el correlograma de la función de autocorrelación (ACF).

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
diferencia = df_train_sma['pasajeros'].diff()

In [ ]:
plt.plot(diferencia)

In [ ]:
# remove the missing values
diferencia.dropna(inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(15,6))
fig = plot_acf(diferencia,lags=200,ax=ax)

**De acuerdo al correlograma se necesita seleccionar al menos 62 lags para hacer predicciones**

In [ ]:
df_train_sma['ajust_62'] = df_train_sma['pasajeros'].rolling(window=62).mean()

In [ ]:
df_train_sma.tail()

In [ ]:
df_test_sma['prediccion_62'] = 64228.661290

In [ ]:
error_62 = df_train_sma['pasajeros'] - df_train_sma['ajust_62']
me_62 = 1.96*error_62.std()

plt.figure(figsize=(15,8))
plt.plot(df_train_sma['mes'],df_train_sma['pasajeros'],'bo:',markerfacecolor='w')

plt.plot(df_train_sma['mes'],df_train_sma['ajust_62'],'s:',color='orange',markerfacecolor='w')


plt.plot(df_test_sma['mes'],df_test_sma['pasajeros'],'go:',markerfacecolor='w')

plt.plot(df_test_sma['mes'],df_test_sma['prediccion_62'],'s:',color='red',markerfacecolor='w')

plt.fill_between(df_test_sma['mes'],(df_test_sma['prediccion_62']+me_62),(df_test_sma['prediccion_62']-me_62),alpha=0.1)
plt.legend(['Datos de entrenamiento','Datos ajustados','Datos de validación','Predicción','Intervalo de confianza'])

start_date = datetime.datetime.strptime("2015-01-01","%Y-%m-%d")
end_date = datetime.datetime.strptime("2018-01-01","%Y-%m-%d")
plt.axis(xmin=start_date,
         xmax=end_date)

plt.show()

## Media móvil ponderada 3 periodos

In [ ]:
df_train_wma = df_train[['mes','pasajeros']]
df_test_wma = df_test[['mes','pasajeros']]

In [ ]:
df_train_wma.head()

In [ ]:
# Ejemplo: Se deben asignar pesos a los valores de la serie de tiempo
# los pesos mayores deben corresponder a las observaciones más recientes
(3 * 58764 + 2 * 48526 + 1 * 46492)/(3+2+1) # -> wma

In [ ]:
def weighted_moving_average(x):
  weights = np.arange(1,len(x)+1,+1)
  num = np.sum(x * weights)
  den = np.sum(weights)
  wma = num / den
  return wma

In [ ]:
ejemplo = df_train_wma.head()

In [ ]:
ejemplo

In [ ]:
(3*58764 + 48526 * 2 +46492)/(6)

In [ ]:
ejemplo['pasajeros'].rolling(window=3).apply(weighted_moving_average)

In [ ]:
# applying to entire data
df_train_wma['ajust_wma_3'] = df_train_wma['pasajeros'].rolling(window=3).apply(weighted_moving_average)
df_train_wma['ajust_wma_62'] = df_train_wma['pasajeros'].rolling(window=62).apply(weighted_moving_average)

In [ ]:
df_train_wma.head()

In [ ]:
df_train_wma.tail()

In [ ]:
df_test_wma['prediccion_3'] = 67778.333333
df_test_wma['prediccion_62'] = 65983.619560

**Gráfica con 62 lags**

In [ ]:
error_62 = df_train_wma['pasajeros'] - df_train_wma['ajust_wma_62']
me_62 = 1.96*error_62.std()

plt.figure(figsize=(15,8))
plt.plot(df_train_wma['mes'],df_train_wma['pasajeros'],'bo:',markerfacecolor='w')

plt.plot(df_train_wma['mes'],df_train_wma['ajust_wma_62'],'s:',color='orange',markerfacecolor='w')


plt.plot(df_test_wma['mes'],df_test_wma['pasajeros'],'go:',markerfacecolor='w')

plt.plot(df_test_wma['mes'],df_test_wma['prediccion_62'],'s:',color='red',markerfacecolor='w')

plt.fill_between(df_test_wma['mes'],(df_test_wma['prediccion_62']+me_62),(df_test_wma['prediccion_62']-me_62),alpha=0.1)
plt.legend(['Datos de entrenamiento','Datos ajustados','Datos de validación','Predicción','Intervalo de confianza'])

# scalling
# start_date = datetime.datetime.strptime("2015-01-01","%Y-%m-%d")
# end_date = datetime.datetime.strptime("2018-01-01","%Y-%m-%d")
# plt.axis(xmin=start_date,
#          xmax=end_date)

plt.show()

## Suavizamiento exponencial

In [ ]:
df_train_ema = df_train[['mes','pasajeros']]
df_test_ema = df_test[['mes','pasajeros']]

In [ ]:
df_train_ema['ajust_3'] = df_train_ema['pasajeros'].ewm(min_periods=3,alpha=0.5).mean()

In [ ]:
df_train_ema.head()

In [ ]:
df_train_ema.tail()

In [ ]:
df_test_ema['prediccion_3'] = 68104.352005

In [ ]:
error_0_3 = df_train_ema['pasajeros'] - df_train_ema['ajust_3']
me_0_3 = 1.96*error_0_3.std()

plt.figure(figsize=(15,8))
plt.plot(df_train_ema['mes'],df_train_ema['pasajeros'],'bo:',markerfacecolor='w')

plt.plot(df_train_ema['mes'],df_train_ema['ajust_3'],'s:',color='orange',markerfacecolor='w')

plt.plot(df_test_ema['mes'],df_test_ema['pasajeros'],'go:',markerfacecolor='w')

plt.plot(df_test_ema['mes'],df_test_ema['prediccion_3'],'s:',color='red',markerfacecolor='w')

plt.fill_between(df_test_ema['mes'],(df_test_ema['prediccion_3']+me_0_3),
                 (df_test_ema['prediccion_3']-me_0_3),alpha=0.1)
plt.legend(['Datos de entrenamiento','Datos ajustados','Datos de validación','Predicción','Intervalo de confianza'])

# start_date = datetime.datetime.strptime("2015-01-01","%Y-%m-%d")
# end_date = datetime.datetime.strptime("2018-01-01","%Y-%m-%d")
# plt.axis(xmin=start_date,
#          xmax=end_date)

plt.show()